In [ ]:
# default_exp clone_counters

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from functools import partial, reduce
from glob import glob
from typing import Callable

import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import xarray as xr
from skimage import measure

from py_clone_detective import clone_analysis as ca
from py_clone_detective.utils import (
    add_scale_regionprops_table_area_measurements,
    calculate_corresponding_labels,
    calculate_overlap,
    check_channels_input_suitable_and_return_channels,
    determine_labels_across_other_images_using_centroids,
    extend_region_properties_list,
    get_all_labeled_clones_unmerged_and_merged,
    img_path_to_xarr,
    last2dims,
    lazy_props,
    reorder_df_to_put_ch_info_first,
    update_1st_coord_and_dim_of_xarr,
)

# CloneCounter Classes

## Parent Class

In [ ]:
# export
class CloneCounter:
    def __init__(
        self,
        exp_name: str,
        img_name_regex: str,
        pixel_size: float,
        tot_seg_ch: str = "C0",
    ):
        self.exp_name = exp_name
        self.img_name_regex = img_name_regex
        self.pixel_size = pixel_size
        self.tot_seg_ch = tot_seg_ch

    def add_images(self, **channel_path_globs):
        return img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="img_channels",
            **channel_path_globs,
        )

    def add_segmentations(
        self,
        additional_func_to_map: Callable = None,
        ad_func_kwargs: dict = None,
        **channel_path_globs,
    ):
        segmentations = img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="seg_channels",
            **channel_path_globs,
        )

        if additional_func_to_map is not None:
            segmentations.data = segmentations.data.map_blocks(
                additional_func_to_map, **ad_func_kwargs, dtype=np.uint16
            )

        segmentations.data = segmentations.data.map_blocks(
            last2dims(partial(measure.label)), dtype=np.uint16
        )
        return segmentations

    def combine_C0_overlaps_and_measurements(self):
        ov_df = (
            self.results_overlaps.pivot(
                index=["img_name", "C0_labels"],
                columns=["colocalisation_ch"],
                values="is_in_label",
            )
            .query("C0_labels != 0")
            .copy()
        )
        sk_df = self.results_measurements.query("seg_ch== 'C0'").set_index(
            ["seg_img", "label"]
        )
        sk_df.index.rename(["img_name", "C0_labels"], inplace=True)
        return pd.merge(ov_df, sk_df, left_index=True, right_index=True)

    def determine_seg_img_channel_pairs(
        self, seg_channels: list = None, img_channels: list = None
    ):
        seg_channels = check_channels_input_suitable_and_return_channels(
            channels=seg_channels,
            available_channels=self.image_data.seg_channels.values.tolist(),
        )

        img_channels = check_channels_input_suitable_and_return_channels(
            channels=img_channels,
            available_channels=self.image_data.img_channels.values.tolist(),
        )

        seg_img_channel_pairs = pd.DataFrame()
        seg_img_channel_pairs["image_channel"] = pd.Series(img_channels)
        seg_img_channel_pairs["segmentation_channel"] = pd.Series(seg_channels)
        self.seg_img_channel_pairs = seg_img_channel_pairs.fillna(method="ffill")[
            ["segmentation_channel", "image_channel"]
        ]

    def make_measurements(
        self,
        seg_channels: list = None,
        img_channels: list = None,
        extra_properties: list = None,
        **kwargs,
    ):

        self.determine_seg_img_channel_pairs(seg_channels, img_channels)

        properties = extend_region_properties_list(extra_properties)

        results = list()
        for _, seg_ch, img_ch in self.seg_img_channel_pairs.itertuples():
            for seg, img in zip(
                self.image_data["segmentations"].loc[seg_ch],
                self.image_data["images"].loc[img_ch],
            ):
                results.append(
                    lazy_props(
                        seg.data,
                        img.data,
                        seg.seg_channels.item(),
                        img.img_channels.item(),
                        seg.img_name.item(),
                        img.img_name.item(),
                        properties,
                        **kwargs,
                    )
                )

        df = dd.from_delayed(results).compute()
        df = add_scale_regionprops_table_area_measurements(df, self.pixel_size)
        self.results_measurements = reorder_df_to_put_ch_info_first(df)
        self._determine_max_seg_label_levels()

    def _determine_max_seg_label_levels(self):
        self.tot_seg_ch_max_labels = (
            self.image_data["segmentations"]
            .loc[self.tot_seg_ch]
            .data.map_blocks(
                lambda x: np.unique(x).shape[0], drop_axis=(1, 2), dtype=np.uint16,
            )
            .compute()
            .max()
        )

    def _create_df_from_arr(self, arr):
        return (
            xr.DataArray(
                np.moveaxis(arr, 1, 0),
                coords=(
                    self.image_data["segmentations"].coords["seg_channels"][1:],
                    self.image_data["segmentations"].coords["img_name"],
                    np.arange(self.tot_seg_ch_max_labels),
                ),
                dims=("colocalisation_ch", "img_name", "C0_labels",),
            )
            .to_dataframe("is_in_label")
            .reset_index()
            .dropna()
        )

    def measure_overlap(self):
        self._determine_max_seg_label_levels()
        arr = (
            self.image_data["segmentations"]
            .data.map_blocks(
                calculate_overlap,
                drop_axis=[0],
                dtype=np.float64,
                num_of_segs=self.image_data["segmentations"].shape[0],
                preallocate_value=self.tot_seg_ch_max_labels,
            )
            .compute()
        )

        df = self._create_df_from_arr(arr)
        df["is_in_label"] = df["is_in_label"].astype(np.uint16)
        self.results_overlaps = df[
            ["img_name", "C0_labels", "colocalisation_ch", "is_in_label"]
        ]

    def filter_labels_update_measurements_df_and_to_dict(
        self, query_for_pd: str, name_for_query: str
    ):

        self.results_measurements[
            f"{name_for_query}_pos"
        ] = self.results_measurements.eval(query_for_pd)
        return (
            self.results_measurements.query(query_for_pd)
            .groupby("int_img")
            .agg({"label": lambda x: list(x)})["label"]
            .to_dict()
        )

    def get_centroids_list(self):
        df = self.results_measurements.query("int_img_ch == @self.tot_seg_ch")
        centroids_list = list()
        for img_name in df["int_img"].unique():
            centroids_list.append(
                (
                    df.query("int_img == @img_name")
                    .loc[:, ["centroid-0", "centroid-1"]]
                    .values.astype(int)
                )
            )
        return centroids_list

    def add_clones_and_neighbouring_labels(
        self,
        query_for_pd: str = 'int_img_ch == "C1" & mean_intensity > 1000',
        name_for_query: str = "C1",
        calc_clones: bool = True,
    ):
        new_coord = [
            "extended_tot_seg_labels",
            "total_neighbour_counts",
            f"{name_for_query}pos_neigh_counts",
            f"{name_for_query}neg_neigh_counts",
        ]

        if calc_clones:
            new_coord.append(f"{name_for_query}_clone")

        clone_coords, clone_dims = update_1st_coord_and_dim_of_xarr(
            self.image_data["images"],
            new_coord=new_coord,
            new_dim=f"{name_for_query}_neighbours",
        )

        labels_to_keep = self.filter_labels_update_measurements_df_and_to_dict(
            query_for_pd, name_for_query
        )

        new_label_imgs = get_all_labeled_clones_unmerged_and_merged(
            self.image_data["segmentations"].loc[self.tot_seg_ch],
            labels_to_keep,
            calc_clones,
        )

        return xr.DataArray(
            data=new_label_imgs,
            coords=clone_coords,
            dims=clone_dims,
            attrs={f"{self.tot_seg_ch}_labels_kept_query": query_for_pd},
        )

    def colabels_to_df(self, colabels, name_for_query):
        return (
            xr.DataArray(
                colabels,
                coords=(
                    self.image_data[name_for_query].coords[
                        f"{name_for_query}_neighbours"
                    ],
                    foo.image_data[name_for_query].coords["img_name"],
                    range(1, colabels.shape[2] + 1),
                ),
                dims=(f"{name_for_query}_neighbours", "img_name", "label"),
            )
            .to_dataframe("colabel")
            .reset_index()
            .dropna()
            .pivot(
                index=["img_name", "label"],
                columns=[f"{name_for_query}_neighbours"],
                values="colabel",
            )
            .astype(np.uint16)
            .query("label == extended_tot_seg_labels")
            .eval(
                f"oth_{name_for_query}pos_neigh_counts = total_neighbour_counts - {name_for_query}neg_neigh_counts"
            )
            .eval(
                f"oth_{name_for_query}neg_neigh_counts = total_neighbour_counts - {name_for_query}pos_neigh_counts"
            )
        )

    def clarify_neighbouring_label_counts(self, name_for_query):
        df = (
            self.results_clones_and_neighbour_counts[name_for_query]
            .assign(
                intermediate_1=lambda x: x[f"oth_{name_for_query}pos_neigh_counts"][
                    x[f"{name_for_query}pos_neigh_counts"] == 0
                ]
            )
            .assign(
                intermediate_2=lambda x: x[f"{name_for_query}pos_neigh_counts"][
                    x[f"oth_{name_for_query}pos_neigh_counts"]
                    == x["total_neighbour_counts"]
                ]
            )
        )
        df[f"{name_for_query}pos_nc"] = df.intermediate_1.fillna(
            0
        ) + df.intermediate_2.fillna(0)
        df[f"{name_for_query}neg_nc"] = (
            df.total_neighbour_counts - df[f"{name_for_query}pos_nc"]
        )

        return (
            df.drop(
                columns=[
                    f"{name_for_query}neg_neigh_counts",
                    f"{name_for_query}pos_neigh_counts",
                    f"oth_{name_for_query}pos_neigh_counts",
                    f"oth_{name_for_query}neg_neigh_counts",
                    "intermediate_1",
                    "intermediate_2",
                ]
            )
            .astype(np.uint16)
            .query(f"{name_for_query}neg_nc != 0 | {name_for_query}pos_nc != 0")
        )

    def measure_clones_and_neighbouring_labels(self, name_for_query):
        self.get_centroids_list()
        colabels = calculate_corresponding_labels(
            self.image_data[name_for_query].data,
            self.get_centroids_list(),
            self.image_data[name_for_query].shape[0],
            foo.tot_seg_ch_max_labels,
        )

        if not hasattr(self, "results_clones_and_neighbour_counts"):
            self.results_clones_and_neighbour_counts = dict()

        self.results_clones_and_neighbour_counts[name_for_query] = self.colabels_to_df(
            colabels, name_for_query
        )

        self.results_clones_and_neighbour_counts[name_for_query].index.rename(
            ["int_img", "label"], inplace=True
        )

        self.results_clones_and_neighbour_counts[
            name_for_query
        ] = self.clarify_neighbouring_label_counts(name_for_query)

    def combine_neighbour_counts_and_measurements(self):
        list_df = list(self.results_clones_and_neighbour_counts.values()) + [
            self.results_measurements.set_index(["int_img", "label"])
        ]
        merged_df = reduce(
            lambda left, right: pd.merge(
                left,
                right,
                how="left",
                on=["int_img", "label"],
                suffixes=(None, "_extra"),
            ),
            list_df,
        )

        cols_to_drop = merged_df.filter(regex="extra").columns.tolist() + [
            "extended_tot_seg_labels"
        ]

        return merged_df.drop(columns=cols_to_drop)

## CloneCounter subclasses

In [ ]:
# export
class LazyCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        )

    def add_segmentations(
        self,
        additional_func_to_map: Callable = None,
        ad_func_kwargs: dict = None,
        **channel_path_globs
    ):
        self.image_data["segmentations"] = super().add_segmentations(
            additional_func_to_map, ad_func_kwargs, **channel_path_globs
        )

    def add_clones_and_neighbouring_labels(
        self,
        query_for_pd: str = 'int_img_ch == "C1" & mean_intensity > 1000',
        name_for_query: str = "filt_C1_intensity",
        calc_clones: bool = True,
    ):
        self.image_data[name_for_query] = super().add_clones_and_neighbouring_labels(
            query_for_pd, name_for_query, calc_clones
        )

In [ ]:
# export
class PersistentCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        ).persist()

    def add_segmentations(
        self,
        additional_func_to_map: Callable = None,
        ad_func_kwargs: dict = None,
        **channel_path_globs,
    ):
        self.image_data["segmentations"] = (
            super()
            .add_segmentations(
                additional_func_to_map, ad_func_kwargs, **channel_path_globs
            )
            .persist()
        )

    def add_clones_and_neighbouring_labels(
        self,
        query_for_pd: str = 'int_img_ch == "C1" & mean_intensity > 1000',
        name_for_query: str = "filt_C1_intensity",
        calc_clones: bool = True,
    ):
        self.image_data[name_for_query] = (
            super()
            .add_clones_and_neighbouring_labels(
                query_for_pd, name_for_query, calc_clones
            )
            .persist()
        )

In [ ]:
# hide
from dask.distributed import Client

c = Client()
c

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:52145,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:52151,Total threads: 2
Dashboard: http://127.0.0.1:52153/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:52147,


## Example using LazyCloneCounter with measure_overlap

In [ ]:
bar = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

bar.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

bar.add_segmentations(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_binaries/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_label_imgs_v2/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_label_imgs/*.tif*",
)
bar.make_measurements(extra_properties=["convex_area"],)
df = bar.measure_overlap()
# df = bar.combine_C0_overlaps_and_measurements()

In [ ]:
ca.query_df_groupby_by_clone_channel(
    df,
    {
        "C2negC3neg_C0area_less_than_50um2": "C3 == 0 & C2 == 0 & area_um2 < 50",
        "C2negC3neg_C0area_greater_than_50um2": "C3 == 0 & C2 == 0 & area_um2 > 50",
        "C2pos": "C2 != 0",
        "C3pos": "C3 != 0",
    },
    clone_channel="C1",
)

C2negC3neg_C0area_less_than_50um2                        \
                                    C0_labels   area_um2              
                                        count       mean        std   
img_name C1                                                           
a1g01p1  0                                 90  23.704236  11.028195   
         1                                  1  31.157500        NaN   
         2                                 14  23.978527   6.857923   
         4                                  9  26.905694  13.324214   
a1g01p2  0                                 93  27.090013  12.270181   
...                                       ...        ...        ...   
a2g12p1  0                                 69  21.032518  12.005780   
a2g12p2  0                                 63  27.849206  13.966935   
a2g13p1  0                                137  28.271054  11.999889   
a2g13p2  0                                 80  26.988672  11.872825   
a2g13p3  0                                 64  23.075078  10.868100   

            C2negC3neg_C0area_greater_than_50um2                         \
                                       C0_labels    area_um2              
                                           count        mean        std   
img_name C1                                                               
a1g01p1  0                                    22   82.469063  33.422793   
         1                                     2   53.504688   0.588224   
         2                                     5   87.377125  37.336913   
         4                                     2  119.298438   7.860818   
a1g01p2  0                                    48   80.080573  20.334236   
...                                          ...         ...        ...   
a2g12p1  0                                    12   71.938281  21.282496   
a2g12p2  0                                    17   59.748199   7.764255   
a2g13p1  0                                    18   57.886736   8.982864   
a2g13p2  0                                     8   58.892969   4.819319   
a2g13p3  0                                     2   52.030000   1.711198   

                C2pos                           C3pos                      
            C0_labels   area_um2            C0_labels  area_um2            
                count       mean        std     count      mean       std  
img_name C1                                                                
a1g01p1  0         81  80.136358  30.521184        29  6.057823  5.057770  
         1          9  94.985000  24.524868         1  5.520625       NaN  
         2         13  71.087500  27.954905         4  3.932500  3.430742  
         4          9  86.237708  37.690737         1  4.461875       NaN  
a1g01p2  0        107  67.489305  32.965702        25  6.107475  4.459860  
...               ...        ...        ...       ...       ...       ...  
a2g12p1  0         95  45.411618  28.140310        26  7.507236  3.430112  
a2g12p2  0         53  35.659328  19.589708        23  8.183940  4.043894  
a2g13p1  0         17  27.692096  16.454961        40  7.190047  4.512039  
a2g13p2  0        108  32.742124  21.818361        35  6.397875  4.275251  
a2g13p3  0        149  47.172236  17.007087        23  8.183940  9.110108  

[114 rows x 12 columns]

## Example using LazyCloneCounter with add_clones_and_neighbouring_labels

In [ ]:
from skimage import morphology

In [ ]:
foo = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

foo.add_segmentations(
    morphology.remove_small_objects,
    ad_func_kwargs={"min_size": 49},
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
)

foo.make_measurements()

In [ ]:
foo.add_clones_and_neighbouring_labels(
    query_for_pd='int_img_ch == "C1" & mean_intensity > 1000',
    name_for_query="C1",
    calc_clones=True,
)

In [ ]:
foo.results_measurements = foo.results_measurements.eval(
    "total_intensity = mean_intensity * area"
)

In [ ]:
foo.add_clones_and_neighbouring_labels(
    query_for_pd='int_img_ch == "C2" & total_intensity > 2e5',
    name_for_query="C2",
    calc_clones=False,
)

In [ ]:
foo.add_clones_and_neighbouring_labels(
    query_for_pd='int_img_ch == "C3" & mean_intensity > 5000',
    name_for_query="C3",
    calc_clones=False,
)

In [ ]:
foo.measure_clones_and_neighbouring_labels(name_for_query="C1")

In [ ]:
foo.measure_clones_and_neighbouring_labels(name_for_query="C2")

In [ ]:
foo.measure_clones_and_neighbouring_labels(name_for_query="C3")

In [ ]:
df = foo.combine_neighbour_counts_and_measurements()

In [ ]:
import napari
view = napari.Viewer()

In [ ]:
view.add_image(foo.image_data['images'], channel_axis=0)

[<Image layer 'Image' at 0x7fbc910cc970>,
 <Image layer 'Image [1]' at 0x7fbc8f103e80>,
 <Image layer 'Image [2]' at 0x7fbc8a101640>,
 <Image layer 'Image [3]' at 0x7fbc914ff070>]

In [ ]:
view.add_labels(foo.image_data['images'][3] > 5000)

<Labels layer 'Labels' at 0x7fbc89980490>

In [ ]:
df.head()

C1_clone  total_neighbour_counts  C1pos_nc  C1neg_nc  C2pos_nc  \
int_img label                                                                   
a1g01p1 12            0                       3         1         2         3   
        12            0                       3         1         2         3   
        12            0                       3         1         2         3   
        12            0                       3         1         2         3   
        13            0                       3         0         3         3   

               C2neg_nc  C3pos_nc  C3neg_nc seg_ch int_img_ch  seg_img  area  \
int_img label                                                                  
a1g01p1 12            0         0         3     C0         C0  a1g01p1  1094   
        12            0         0         3     C0         C1  a1g01p1  1094   
        12            0         0         3     C0         C2  a1g01p1  1094   
        12            0         0         3     C0         C3  a1g01p1  1094   
        13            0         0         3     C0         C0  a1g01p1  1123   

               mean_intensity  centroid-0  centroid-1   area_um2  C1_pos  \
int_img label                                                              
a1g01p1 12        2103.588665   43.901280   89.606947  82.733750   False   
        12          60.372029   43.901280   89.606947  82.733750   False   
        12        2209.449726   43.901280   89.606947  82.733750   False   
        12          73.462523   43.901280   89.606947  82.733750   False   
        13        3414.680321   51.479964  362.270703  84.926875   False   

               total_intensity  C2_pos  C3_pos  
int_img label                                   
a1g01p1 12           2301326.0   False   False  
        12             66047.0   False   False  
        12           2417138.0    True   False  
        12             80368.0   False   False  
        13           3834686.0   False   False

In [ ]:
ca.query_df_groupby_by_clone_channel(
    df,
    queries={
        "C2negC3neg_C0area_less_than_50um2": "C2_pos == False & C3_pos == False & area_um2 < 50",
        "C2negC3neg_C0area_greater_than_50um2": "C2_pos == False & C3_pos == False & area_um2 > 50"},
    clone_channel="C1_clone"
)

KeyError: "Column(s) ['C0_labels'] do not exist"